In [ ]:
%pylab inline
import pandas as pd
import pysumma as ps
import xarray as xr
from matplotlib import cm
import seaborn as sns
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from statsmodels.api import OLS
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
%run ../lib/summa_snow_layering.py

sns.set_context('poster')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)

S_PER_HOUR = 3600

In [ ]:
dana = xr.open_dataset('../processed/dana_perturbations.nc').load()
coldeport = xr.open_dataset('../processed/coldeport_perturbations.nc').load()
reynolds = xr.open_dataset('../processed/reynolds_perturbations.nc').load()

dana['scalarSnowTemp'] -= 273.16
reynolds['scalarSnowTemp'] -= 273.16
coldeport['scalarSnowTemp'] -= 273.16

dana['scalarSurfaceTemp'] -= 273.16
reynolds['scalarSurfaceTemp'] -= 273.16
coldeport['scalarSurfaceTemp'] -= 273.16

In [ ]:
dana_years = np.arange(dana.time.dt.year.values[0], dana.time.dt.year.values[-1]-1)
cdp_years = np.arange(coldeport.time.dt.year.values[0], coldeport.time.dt.year.values[-1])
reynolds_years = np.arange(reynolds.time.dt.year.values[0], reynolds.time.dt.year.values[-1])

In [ ]:
colors_2l = ['wheat', 'orange', 'peru']
colors_3l = ['skyblue', 'dodgerblue', 'royalblue']
colors_4l = ['violet', 'deeppink', 'crimson']
colors_all = [ *colors_2l, *colors_3l, *colors_4l, 'lime']

In [ ]:
year = lambda x, y: slice('{}/10/01'.format(x), '{}/09/30'.format(y))

In [ ]:
year_dict = {'dana': dana_years,
             'cdp': cdp_years,
             'reynolds': reynolds_years}

ds_dict = {'dana': dana,
           'cdp': coldeport,
           'reynolds': reynolds}

In [ ]:
sites = ['dana', 'cdp', 'reynolds']
temps = ['-2.0K', '+0.0K', '+2.0K', '+4.0K']
layers = ['2L_thin' , '2L_mid', '2L_thick', '3L_thin', '3L_mid', '3L_thick', '4L_thin', '4L_mid', '4L_thick', 'CLM-like']

site_years = [f'{s}_{y}' for s in sites for y in year_dict[s][:-1]]
site_model_years = [f'{s}_{m}_{y}' for s in sites for m in layers for y in year_dict[s][:-1]]
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=c, label=m, markersize=15) for m, c in zip(layers, colors_all)]
layers = ['2L_thin' , '2L_mid', '2L_thick', '3L_thin', '3L_mid', '3L_thick', '4L_thin', '4L_mid', '4L_thick', 'CLM']

In [ ]:
met_snow = {s: {dt: [] for dt in temps} for s in sites}
for site in ['dana', 'cdp', 'reynolds']:
    for dt in temps:
        for y in year_dict[site][1:-1]:
                tds = ds_dict[site]['scalarSnowfall'].sel(time=year(y, y+1), dt=dt, model='JRDN')
                tbs = np.diff(np.argwhere((tds > 0).values).flatten())
                tbs = tbs[tbs > 1]
                met_snow[site][dt].append(tbs)

In [ ]:
mbe_swe = {s: {l: {dt: [] for dt in temps} for l in layers} for s in sites }
mbe_cc  = {s: {l: {dt: [] for dt in temps} for l in layers} for s in sites }
mbe_sst = {s: {l: {dt: [] for dt in temps} for l in layers} for s in sites }
met_snow = {s: {l: {dt: [] for dt in temps} for l in layers} for s in sites }
site_year_strings = []

for site in ['dana', 'cdp', 'reynolds']:
    for dt in temps[0:2]:
        for layer in layers:
            for y in year_dict[site][1:-1]:
                sim_swe   = ds_dict[site]['scalarSWE'].sel(time=year(y, y+1), dt=dt, model=layer)
                jrdn_swe  = ds_dict[site]['scalarSWE'].sel(time=year(y, y+1), dt=dt, model='JRDN')
                sim_sst   = ds_dict[site]['scalarSurfaceTemp'].sel(time=year(y, y+1), dt=dt, model=layer)
                jrdn_sst  = ds_dict[site]['scalarSurfaceTemp'].sel(time=year(y, y+1), dt=dt, model='JRDN')
                sim_cc    = ds_dict[site]['scalarColdContent'].sel(time=year(y, y+1), dt=dt, model=layer)
                jrdn_cc   = ds_dict[site]['scalarColdContent'].sel(time=year(y, y+1), dt=dt, model='JRDN')
                sim_sf    = ds_dict[site]['scalarSnowfall'].sel(time=year(y, y+1), dt=dt, model=layer)
                
                peaksnow = int(sim_swe.argmax().values[()])
                firstsnow = np.where(sim_swe.isel(time=slice(0, peaksnow)) != 0)[0][0]
                nosnow = np.where(sim_swe.isel(time=slice(peaksnow, None)) == 0)[0]
                if len(nosnow):
                    nosnow = nosnow[0] + peaksnow
                else:
                    nosnow = None
                melt_season = slice(peaksnow, nosnow)
                accum_season = slice(firstsnow, peaksnow)
                snow_season = slice(firstsnow, nosnow)
                sim_swe   = sim_swe.isel(time=melt_season).values
                jrdn_swe  = jrdn_swe.isel(time=melt_season).values
                
                sim_cc = sim_cc.isel(time=accum_season).values / 1e6
                jrdn_cc = jrdn_cc.isel(time=accum_season).values / 1e6
                
                sim_sst = sim_sst.isel(time=accum_season).values
                jrdn_sst = jrdn_sst.isel(time=accum_season).values
                
                mask_cc = np.logical_and(~np.isnan(sim_cc), ~np.isnan(jrdn_cc))
                met_snow[site][layer][dt].append(np.sum(sim_sf > 0) / len(sim_sf))
                mbe_swe[site][layer][dt].append(mbe(sim_swe, jrdn_swe))
                mbe_cc[site][layer][dt].append( mbe(sim_cc[mask_cc], jrdn_cc[mask_cc]))
                mbe_sst[site][layer][dt].append(mbe(sim_sst, jrdn_sst))
                

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 12)
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False, sharex=False)
site_name = ['Dana Meadows', 'Col de Porte', 'Reynolds Creek']
for k, s in enumerate(sites):
    for i, layer in enumerate(layers):
        for j, dt in enumerate(temps):
            met_list = mbe_swe[s][layer][dt]
            sf_list = mbe_sst[s][layer][dt]
            axes[k].scatter(sf_list, met_list, color=colors_all[i], alpha=0.8)
            axes[k].set_title(site_name[k])
axes[0].set_ylabel('')
axes[1].set_xlabel('')
plt.legend(handles=legend_elements, bbox_to_anchor=(1.1, 1.))

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 12)
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False, sharex=False)
site_name = ['Dana Meadows', 'Col de Porte', 'Reynolds Creek']
for k, s in enumerate(sites):
    for i, layer in enumerate(layers):
        for j, dt in enumerate(temps):
            met_list = mbe_swe[s][layer][dt]
            sf_list = mbe_cc[s][layer][dt]
            axes[k].scatter(sf_list, met_list, color=colors_all[i], alpha=0.8)
            axes[k].set_title(site_name[k])
axes[0].set_ylabel(r'$\Delta$SWE from SNTHERM-like (mm)')
axes[1].set_xlabel(r'$\Delta$ Cold Content from SNTHERM-like (MJ)')
plt.legend(handles=legend_elements, bbox_to_anchor=(1.1, 1.))

In [ ]:
fig, axes = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(12, 12))
mbe_cc_list = []
mbe_swe_list = []
for k, s in enumerate(sites[0:2]):
    for i, l in enumerate(layers):
        for j, dt in enumerate(temps):
            axes.scatter(mbe_cc[s][l][dt],
                         mbe_swe[s][l][dt], color=colors_all[i],
                         alpha=0.7)
            mbe_cc_list.append(np.array(mbe_cc[s][l][dt]))
            mbe_swe_list.append(np.array(mbe_swe[s][l][dt]))
       
mbe_cc_list = np.hstack(mbe_cc_list).reshape(-1, 1)
mbe_swe_list = np.hstack(mbe_swe_list).reshape(-1, 1)

ols = OLS(mbe_swe_list, mbe_cc_list, )
res = ols.fit(cov_type='HC1')

res.params[0]
res.rsquared_adj
res.f_pvalue

fig.text(0.5, 0.04, r'$Cold\ content$ (MJ)', ha='center', fontsize=22)
fig.text(0.0, 0.5, r'SWE (mm)', va='center', rotation='vertical', fontsize=22)
plt.legend(handles=legend_elements, bbox_to_anchor=(1., 1.))
